# Step 1: fine-tune LLM using top result from (fixed) ranker

In [ ]:
def launch():
    import os
    import StepOne
    import torch

    os.environ['MODEL_TYPE'] = 'xxl'
    os.environ['BATCH_SIZE'] = '1'
    os.environ['AUGMENT'] = os.environ.get('AUGMENT', '2')
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepOne.step_one,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 2 max_iteration = 5 model_type = xxl *********
n              iter (since)      4 loss (since)       4 acc (since)   4 ema acc (since) 4 acc (dev) (since)      dt
1             0.000 (0.000)       1.058 (1.058)       0.750 (0.750)       0.750 (0.750)       0.000 (0.000)  3.29 s
2             0.000 (0.000)       0.714 (0.371)       0.750 (0.750)       0.875 (1.000)       0.000 (0.000)  7.27 s
4             0.000 (0.000)       3.102 (5.490)       0.688 (0.625)       0.875 (0.875)       0.000 (0.000)  13.7 s
8             0.000 (0.000)       2.223 (1.344)       0.656 (0.625)       0.906 (0.938)       0.000 (0.000)  26.5 s
16            0.000 (0.000)       1.520 (0.817)       0.703 (0.750)       0.812 (0.719)       0.000 (0.000)  54.8 s
32            0.000 (0.000)       1.443 (1.365)       0.680 (0.656)       0.758 (0.703)       0.000 (0.000)  1.83 m
64            0.000 (0.000)       1.250 (1.057)       0.660 (0.641)       0.711 (0.664)       0.000 (0.000)  3.64 m
128   

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def launch():
    import os
    import StepTwo
    import torch

    os.environ['MODEL_TYPE'] = 'xxl'
    os.environ['BATCH_SIZE'] = '2'
    os.environ['LEARN_BATCH_SIZE'] = '2'
    os.environ['STEP1_ITER'] = '0_augment2'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepTwo.step_two,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 2 max_iteration = 5 model_type = xxl *********
n              iter (since)      4 loss (since)       4 acc (since)   4 acc ema (since) 4 acc (dev) (since)      nsamps (since)      dt
1             0.000 (0.000)       0.075 (0.075)       1.000 (1.000)       1.000 (1.000)       0.000 (0.000)     140.000 (140.000)  2.79 m
2             0.000 (0.000)       0.055 (0.035)       1.000 (1.000)       1.000 (1.000)       0.000 (0.000)      72.000 (4.000)  3.81 m
4             0.000 (0.000)       0.027 (0.000)       1.000 (1.000)       1.000 (1.000)       0.000 (0.000)      36.875 (1.750)  6.13 m
8             0.000 (0.000)       0.014 (0.000)       1.000 (1.000)       1.000 (1.000)       0.000 (0.000)      19.812 (2.750)  10.4 m
16            0.000 (0.000)       0.361 (0.709)       0.875 (0.750)       0.875 (0.750)       0.000 (0.000)      13.812 (7.812)  17.7 m
32            0.000 (0.000)       0.510 (0.659)       0.781 (0.688)       0.812 (0.750)       0.000 (0.000)      15.

# Step 3: Prepare Submission Files

In [ ]:
def prepare_submission_probensemble(*, nvoters, step2_iter, step1_iter, k):
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedNewsCat import dev_loader, test_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 0
    torch.set_default_device(device)
    torch.manual_seed(8675309)

    dev = dev_loader(batch_size=8)
    test = test_loader(batch_size=8)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-xxl', load_in_8bit=True, device_map=0)
    taskllm_model_id = f'User_keq{k}_t5xxl_step1_iter{step1_iter}'
    t5.load_adapter(taskllm_model_id, 'raw_taskllm')
    t5.load_adapter(taskllm_model_id, 'ema_taskllm')
    rhat_model_id = f'User_keq{k}_t5xxl_step2_iter{step2_iter}'
    t5.load_adapter(rhat_model_id, 'raw_rhat')
    t5.load_adapter(rhat_model_id, 'ema_rhat')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_suffix="taskllm", model_id=taskllm_model_id, choices=dev.choices)
    rewardpredictor = RewardPredictor(t5=t5, adapter_suffix="rhat", model_id=rhat_model_id)

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(embeddings, nsamples):
        scores = embeddings[0,:] @ embeddings[1:,:].T
        temperature = scores[0].item() - scores[min(scores.shape[0]-1, 4)].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape).to(scores.device)
        safek = min(k, scores.shape[0])
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=safek).indices, sorted=False, dim=0)

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]

    def make_prior(profile):
        from math import log

        c = [1]*len(dev.choices)
        for v in profile:
            c[dev.choices.index(v['category'])] += 1
        n = sum(c)

        return [ log(cnt) - log(n) for cnt in c ]
    
    print(f'*** step1_iter: {step1_iter} step2_iter: {step2_iter} nvoters {nvoters} ***')

    devgolds, testgolds = [], []
    with ProgressPrinter(f'{k} acc (dev)') as printer:
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for isdev, (examples, labels) in interleave(dev, test, sequential=True):
            with torch.no_grad():
                prior = [ make_prior(ex['profile']) for ex in examples ]
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['article'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['text'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = 128,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                randos = [ randomized_similarity(embeddings[a:b,:], 64) for a, b in zip(splits, splits[1:]) ]
                prompts = [ [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in indices ])
                              for indices in rando.to('cpu').tolist()
                            ]
                            for ex, rando in zip(examples, randos)
                          ]
                rhats = torch.cat(inner_batch(func = rewardpredictor.predict,
                                              inner_batch_size = 128,
                                              inputs = (sum(prompts, []),)
                                             ),
                                  dim=0)
                splits = cumsum(map(len, prompts))
                votingprompts = [ [ prompt[v] for v in torch.topk(rhats[a:b].view(-1), k=min(nvoters, b-a)).indices.to('cpu').tolist() ]
                                    for a, b, prompt in zip(splits, splits[1:], prompts)
                                ]
                votingpriors = [ [q]*min(nvoters, b-a) for a, b, q in zip(splits, splits[1:], prior) ]
                predicts = torch.cat(inner_batch(func = lambda p, q: taskllm.predict(p, prior=torch.Tensor(q).to(device)),
                                                 inner_batch_size = 128,
                                                 inputs = (sum(votingprompts, []), sum(votingpriors, [])),
                                                ),
                                     dim=0)
                splits = cumsum(map(len, votingprompts))
                guesses = torch.cat([ predicts[a:b,:].logsumexp(dim=0, keepdim=True).argmax(dim=1)
                                      for a, b in zip(splits, splits[1:])
                                    ],
                                    dim=0)

                if isdev:
                    targets = [ dev.choices.index(label) for label in labels ]
                    targets = torch.Tensor(targets).long().to(guesses.device)
                    acc = (guesses == targets).float().mean().item()
                else:
                    acc = None

                for ex, guess in zip(examples, guesses):
                    (devgolds if isdev else testgolds).append({ 'id': ex['id'], 'output': dev.choices[guess] })

            printer.addobs(acc)

    for wut, golds in ( ('dev', devgolds), ('test', testgolds) ):
        with open(f'lamp2u_{wut}golds_t5xxl_keq{k}_step1_iter{step1_iter}_step2_iter{step2_iter}_nvoters{nvoters}.json', 'w') as jsonfile:
            json.dump({ 'task': 'LaMP_2', 'golds': golds }, jsonfile)

for nvoters in [1, 3, 5, 7]:
    # ugh ... without complete cleanup, run out of memory
    from multiprocessing import Process
    p = Process(target=prepare_submission_probensemble, 
                kwargs = { 'k': 4,
                           'step1_iter': '0_augment2', 
                           'step2_iter': '0_augment2', 
                           'nvoters': nvoters
                         })
    p.start()
    p.join()

*** step1_iter: 0_augment2 step2_iter: 0_augment2 nvoters 1 ***
n       4 acc (dev) (since)      dt
1             0.750 (0.750)  36.2 s
2             0.875 (1.000)  1.09 m
4             0.875 (0.875)  1.93 m
8             0.891 (0.906)  3.72 m
16            0.891 (0.891)  7.23 m
32            0.891 (0.891)  14.8 m
64            0.857 (0.824)  30.1 m
128           0.862 (0.867)  1.02 h
256           0.865 (0.938)   2.1 h
*** step1_iter: 0_augment2 step2_iter: 0_augment2 nvoters 3 ***
n       4 acc (dev) (since)      dt
1             0.750 (0.750)  37.3 s
2             0.812 (0.875)  1.12 m
4             0.812 (0.812)  1.98 m
8             0.859 (0.906)  3.83 m
16            0.883 (0.906)  7.46 m
32            0.887 (0.891)  15.3 m
64            0.857 (0.828)  31.1 m
128           0.861 (0.865)  1.05 h
256           0.864 (0.938)  2.17 h
*** step1_iter: 0_augment2 step2_iter: 0_augment2 nvoters 5 ***
n       4 acc (dev) (since)      dt
1             0.750 (0.750)  38.5 s
2             0.

> Hi,
> 
> This is the results of your latest submission to the LaMP benchmark:
> 
> {"accuracy": 0.7880690737833596, "f1": 0.6566988264536381}